# Import

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
import imblearn
import statistics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# check version number
import imblearn
print(imblearn.__version__)

0.7.0


# Create Dataframe

In [43]:
pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment = None

df = pd.read_csv("NonPolar.csv")
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'fips', 'state', 'county', 'wins',
       'poverty_rate_1960', 'poverty_rate_1970', 'poverty_rate_1980',
       'poverty_rate_1990',
       ...
       'percent_black_male_2019', 'percent_black_female_2019',
       'percent_native_american_male_2019',
       'percent_native_american_female_2019', 'percent_asian_male_2019',
       'percent_asian_female_2019', 'percent_hispanic_male_2019',
       'percent_hispanic_female_2019', 'cumulative_confirmed_cases',
       'cumulative_deaths'],
      dtype='object', length=173)

In [44]:
binary_bellwethers = []
for i, v in enumerate(df['wins']):
    if df.iloc[i, 5] == 5 or df.iloc[i, 5] == 6:
        binary_bellwethers.append(1)
    elif df.iloc[i, 5] < 5:
        binary_bellwethers.append(0)
        
df = df.drop(["Unnamed: 0", "fips", "state", "county", "wins","Unnamed: 0.1"], axis=1)

df.insert(0, 'bellwether?', binary_bellwethers)

In [45]:
df.head()

bellwether?  poverty_rate_1960  poverty_rate_1970  poverty_rate_1980  \
0            0               62.1               42.1               31.5   
1            0               35.4               22.4               14.5   
2            0               63.7               45.0               29.4   
3            0               52.1               24.4               15.7   
4            0               30.9               18.8               15.3   

   poverty_rate_1990  poverty_rate_2000  poverty_rate_2010  population_1960  \
0               30.2               24.5               20.2            18073   
1               14.6               14.0               16.2            46794   
2               29.7               26.6               30.4            18081   
3               16.6               13.7               16.2            37307   
4               16.0               14.8               17.0           629388   

   population_1970  population_1980  population_1990  population_2000  \
0            16515            16793            15929            15738   
1            49352            54165            51345            54384   
2            15545            15782            13939            13987   
3            39127            51132            47552            53333   
4           638855           662694           639105           647628   

   population_2010  persons_in_poverty_1960  persons_in_poverty_1970  \
0            13668                    11225                     6958   
1            53890                    16559                    11076   
2            13119                    11515                     7000   
3            52693                    19437                     9530   
4           645062                   194321                   119976   

   persons_in_poverty_1980  persons_in_poverty_1990  persons_in_poverty_2000  \
0                     5285                     4809                     3858   
1                     7869                     7477                     7592   
2                     4642                     4145                     3719   
3                     8052                     7901                     7293   
4                   101520                   102276                    95674   

   persons_in_poverty_2010  rural_urban_continuum_code_2013  \
0                     2758                                6   
1                     8735                                9   
2                     3982                                7   
3                     8533                                3   
4                   109935                                8   

   urban_influence_code_2013  metro_2013  civilian_labor_force_2000  \
0                          6           0                       9156   
1                         10           0                       6184   
2                         11           0                      11597   
3                          2           1                      12180   
4                          6           0                       3510   

   civilian_labor_force_2001  civilian_labor_force_2002  \
0                       9003                       9144   
1                       5869                       5731   
2                      11019                      10718   
3                      12047                      11709   
4                       3494                       3522   

   civilian_labor_force_2003  civilian_labor_force_2004  \
0                       9183                       9088   
1                       5651                       5462   
2                      10769                      10391   
3                      11722                      11688   
4                       3504                       3484   

   civilian_labor_force_2005  civilian_labor_force_2006  \
0                       9139                       9238   
1                       5300                       5329   
2                      10126   

In [46]:
df.groupby('bellwether?').size()

bellwether?
0    570
1     56
dtype: int64

In [72]:
features = df.drop(['bellwether?'], axis=1)

# Create Principal Components

In [48]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [49]:
#standardization, PCA pipelining

features_df_raw = features
features_df_log = np.log(features_df_raw + 1)

samples = features_df_log.values
scaler = StandardScaler()
pca = PCA()
pipeline = make_pipeline(scaler, pca)
pipeline.fit(samples)

std_val = StandardScaler().fit_transform(features_df_log)
pca = PCA()
principalComponents = pca.fit_transform(std_val)

In [50]:
PCA_components = pd.DataFrame(principalComponents)
PCA_components


0          1          2          3          4          5    \
0    -2.694183  -7.661438   2.434343  -3.751954   4.329655  -1.425980   
1    -4.766801  -2.766472   6.255390  -1.775647   0.291033   0.525349   
2    -1.391390  -6.948368   5.657152  -5.362060   3.180858   1.594653   
3    -0.266228   1.418868   2.768204  -1.606528   2.007991  -1.758616   
4    -6.221278  -7.465476  11.200364   4.203774   3.217434  -1.917989   
5     6.531149   0.251054  -0.599539  -2.254340   4.235030  -0.222683   
6    10.075031   3.473045   2.568853  -4.019910   8.629090   1.737958   
7     6.309566  -6.272881   3.788199  -4.072819   0.387441   2.501252   
8    10.916391   1.352221   0.059373  -3.647745   4.105612   3.674316   
9    -2.927851  -6.304850   1.716056  -1.878199   0.689020  -0.140925   
10   -0.075722  -1.149611  -0.961547  -2.823876  -1.017139  -1.337736   
11   -1.575033  -0.056763  -0.511172   2.937421   2.796917  -1.393452   
12   -5.311812  -8.544785   0.533173  -1.579315   2.089691  -2.338497   
13   -3.225902  -4.420459  -1.573749  -2.353233   4.309707  -0.094358   
14   -2.550059  -3.769302   0.796102  -2.046164   2.660184   0.199417   
15   -8.989388  -5.324400   0.620356  -0.331826  -1.851118  -1.875734   
16   -5.667354  -7.464231   2.374368  -1.142205   2.280657   1.259592   
17   -3.533800  -4.746964   1.624899  -1.419862  -0.561098   0.514650   
18   -8.374816  -3.441536  -3.184149  -1.671526  -0.202491  -1.934064   
19    6.133051  -0.430332   0.261492  -3.183571   2.989511  -0.567576   
20   -3.263504  -3.553092  -1.910580  -2.393960   4.782112  -0.894972   
21   -1.197866  -2.214633  -2.123762  -2.257438   0.120986  -2.756986   
22   -0.177873  -4.141192  -0.693782  -2.511512   3.496322  -0.308071   
23   -7.288345  -5.063365  -0.434861  -1.665256   1.018472   0.376653   
24   -8.853015  -9.459260   3.859569   0.701991   4.193730  -0.320563   
25   -7.281674  -7.792788  -0.067960  -1.872561   4.934561  -1.008913   
26   -3.853301  -4.468735  -1.955875  -1.762827   4.038543  -1.487157   
27   -5.467974  -2.205613  -3.078439   1.986033   7.198023  -3.115580   
28   -9.501549  -6.196728   0.241320  -0.272468   1.448676  -2.291867   
29   -7.842437  -3.762043  -2.618294  -0.690174  -0.429036  -2.360485   
30    7.842445   0.530867  -1.498814  -4.808888   5.380024  -0.500952   
31   -0.147383  -0.740665  -3.739738  -9.724727  -0.565902   5.818901   
32   11.690662   4.828371   0.345686   3.732694   4.007506   0.115394   
33  -18.162063  -0.762601   8.122872  11.937060  -2.719402   9.951244   
34   10.188383  -6.126869   8.526644   2.184539  -1.618131   5.199672   
35   -1.387090  -0.811423   1.431883  -3.625798  -5.073393   5.481440   
36   14.623016  11.124162   1.404724  -1.820236   1.080346   3.963040   
37   -1.274079   0.320287   9.412293  12.103919  -3.036198   5.329995   
38   -6.064179  -3.525229   9.894604   9.066640  -3.275585   5.402994   
39   11.655730   4.994334   6.582494   8.572725   3.254358   3.379337   
40    9.337529   4.363416   8.135596  10.135071   0.104129   4.411458   
41   16.733603   5.031435   5.950678   6.766167   2.784788   5.233494   
42   -1.142959  -2.051402   7.518643   2.694439  -5.978293   6.487773   
43   17.986767   5.050540   5.978516   3.418455   2.559782   3.844191   
44   10.770232   0.204565   1.794177  -6.453611  -3.203493   4.838632   
45   18.075533   3.555002   6.489688   4.752744  -0.805240   2.901116   
46   -5.763020  -3.394475   4.064816  -2.571189  -4.357349   8.416348   
47   18.075357  -6.509947   9.844713   1.117118  -2.589528   2.579794   
48   16.533009   3.601403  -1.405111  -4.842848  -0.592111  -0.078815   
49   18.492037   9.197887  -2.084266  -7.070773  -1.491769   3.214329   
50   -7.097654   2.354177  -3.113604  -0.359483  -0.090396   7.136042   
51   -9.121010  -1.945205  -0.068869   0.772630  -3.576346   2.793576   
52   12.879944  10.180067  -4.788417  -2.955625   0.249904   7.105069   
53   -1.880472   6.104811  -4.984401  -0.739673   0.574029   5.946863

In [84]:
pca_data = {'attributes': list(features.columns), 'first_pc': (abs(principalComponents))[0], 'second_pc': (abs(principalComponents))[1], 'third_pc': (abs(principalComponents))[2], 'fourth_pc': (abs(principalComponents))[3]}
pca_data_df = pd.DataFrame(data=pca_data)
pca_data_df

attributes  first_pc  second_pc  \
0                                  poverty_rate_1960  2.694183   4.766801   
1                                  poverty_rate_1970  7.661438   2.766472   
2                                  poverty_rate_1980  2.434343   6.255390   
3                                  poverty_rate_1990  3.751954   1.775647   
4                                  poverty_rate_2000  4.329655   0.291033   
5                                  poverty_rate_2010  1.425980   0.525349   
6                                    population_1960  0.567190   1.040445   
7                                    population_1970  1.187856   1.441041   
8                                    population_1980  2.330276   0.237542   
9                                    population_1990  0.721256   0.247925   
10                                   population_2000  0.072287   0.667125   
11                                   population_2010  0.811764   1.978664   
12                           persons_in_poverty_1960  0.083767   1.036477   
13                           persons_in_poverty_1970  0.140101   0.701951   
14                           persons_in_poverty_1980  1.719278   0.605252   
15                           persons_in_poverty_1990  1.357452   0.228147   
16                           persons_in_poverty_2000  1.221437   0.490510   
17                           persons_in_poverty_2010  1.269629   0.142963   
18                   rural_urban_continuum_code_2013  0.074002   0.005680   
19                         urban_influence_code_2013  0.803168   0.852439   
20                                        metro_2013  0.020890   0.075213   
21                         civilian_labor_force_2000  0.118463   0.232738   
22                         civilian_labor_force_2001  0.162325   1.095091   
23                         civilian_labor_force_2002  0.110347   0.949674   
24                         civilian_labor_force_2003  0.384062   0.090762   
25                         civilian_labor_force_2004  0.764969   0.665199   
26                         civilian_labor_force_2005  0.703656   0.526194   
27                         civilian_labor_force_2006  0.473101   0.667784   
28                         civilian_labor_force_2007  0.117966   0.281774   
29                         civilian_labor_force_2008  0.636223   0.454516   
30                         civilian_labor_force_2009  1.186495   0.870865   
31                         civilian_labor_force_2010  0.043798   0.121756   
32                         civilian_labor_force_2011  0.728830   0.009572   
33                         civilian_labor_force_2012  0.132689   0.087218   
34                         civilian_labor_force_2013  0.202745   0.302432   
35                         civilian_labor_force_2014  0.452137   0.020826   
36                         civilian_labor_force_2015  0.370095   0.160169   
37                         civilian_labor_force_2016  0.109221   0.434639   
38                         civilian_labor_force_2017  0.521508   0.675041   
39                         civilian_labor_force_2018  0.339373   0.270331   
40                         civilian_labor_force_2019  0.036406   0.103100   
41                                     employed_2000  0.351827   0.010347   
42                                     employed_2001  0.041916   0.231179   
43                                     employed_2002  0.020917   0.011358   
44                                     employed_2003  0.096277   0.032563   
45                                     employed_2004  0.015098   0.011830   
46                                     employed_2005  0.023371   0.207681   
47                                     employed_2006  0.132265   0.150296   
48                                     employed_2007  0.146730   0.527751   
49                                     employed_2008  0.133960   0.167382   
50                                     employed_2009  0.171772   0.009267   
51                                    

# Using PCA to See Attribute Significance

In [88]:
print(pca_data_df[pca_data_df == pca_data_df.max()])
# in PC 1, 3 -> poverty_rate_1970 is most important
# in PC 2, 4 -> poverty_rate_1980 is most important

                    attributes  first_pc  second_pc  third_pc  fourth_pc
0                          NaN       NaN        NaN       NaN        NaN
1                          NaN  7.661438        NaN  6.948368        NaN
2                          NaN       NaN    6.25539       NaN   2.768204
3                          NaN       NaN        NaN       NaN        NaN
4                          NaN       NaN        NaN       NaN        NaN
5                          NaN       NaN        NaN       NaN        NaN
6                          NaN       NaN        NaN       NaN        NaN
7                          NaN       NaN        NaN       NaN        NaN
8                          NaN       NaN        NaN       NaN        NaN
9                          NaN       NaN        NaN       NaN        NaN
10                         NaN       NaN        NaN       NaN        NaN
11                         NaN       NaN        NaN       NaN        NaN
12                         NaN       NaN        NaN

In [110]:
first_pca_df = pca_data_df[["attributes", "first_pc"]]
first_pca_df.sort_values("first_pc", ascending=False)

attributes  first_pc
1                                  poverty_rate_1970  7.661438
4                                  poverty_rate_2000  4.329655
3                                  poverty_rate_1990  3.751954
0                                  poverty_rate_1960  2.694183
2                                  poverty_rate_1980  2.434343
8                                    population_1980  2.330276
14                           persons_in_poverty_1980  1.719278
5                                  poverty_rate_2010  1.425980
15                           persons_in_poverty_1990  1.357452
17                           persons_in_poverty_2010  1.269629
16                           persons_in_poverty_2000  1.221437
7                                    population_1970  1.187856
30                         civilian_labor_force_2009  1.186495
11                                   population_2010  0.811764
19                         urban_influence_code_2013  0.803168
25                         civilian_labor_force_2004  0.764969
32                         civilian_labor_force_2011  0.728830
9                                    population_1990  0.721256
26                         civilian_labor_force_2005  0.703656
29                         civilian_labor_force_2008  0.636223
6                                    population_1960  0.567190
38                         civilian_labor_force_2017  0.521508
27                         civilian_labor_force_2006  0.473101
35                         civilian_labor_force_2014  0.452137
24                         civilian_labor_force_2003  0.384062
36                         civilian_labor_force_2015  0.370095
41                                     employed_2000  0.351827
39                         civilian_labor_force_2018  0.339373
64                                   unemployed_2003  0.294112
65                                   unemployed_2004  0.247189
59                                     employed_2018  0.233179
34                         civilian_labor_force_2013  0.202745
62                                   unemployed_2001  0.179664
57                                     employed_2016  0.179058
50                                     employed_2009  0.171772
22                         civilian_labor_force_2001  0.162325
51                                     employed_2010  0.152610
48                                     employed_2007  0.146730
13                           persons_in_poverty_1970  0.140101
49                                     employed_2008  0.133960
55                                     employed_2014  0.133190
33                         civilian_labor_force_2012  0.132689
47                                     employed_2006  0.132265
63                                   unemployed_2002  0.118769
21                         civilian_labor_force_2000  0.118463
28                         civilian_labor_force_2007  0.117966
23                         civilian_labor_force_2002  0.110347
37                         civilian_labor_force_2016  0.109221
67                                   unemployed_2006  0.104702
75                                   unemployed_2014  0.103510
44                                     employed_2003  0.096277
71                                   unemployed_2010  0.092353
68                                   unemployed_2007  0.088480
12                           persons_in_poverty_1960  0.083767
79                                   unemployed_2018  0.082167
54                                     employed_2013  0.082015
83                            unemployment_rate_2002  0.075108
18                   rural_urban_continuum_code_2013  0.074002
10                                   population_2000  0.072287
56                                     employed_2015  0.064777
87                            unemployment_rate_2006  0.064595
80                                   unemployed_2019  0.063216
58                                     employed_2017  0.062931
60        

In [113]:
second_pca_df = pca_data_df[["attributes", "second_pc"]]
second_pca_df.sort_values("second_pc", ascending=False)

attributes  second_pc
2                                  poverty_rate_1980   6.255390
0                                  poverty_rate_1960   4.766801
1                                  poverty_rate_1970   2.766472
11                                   population_2010   1.978664
3                                  poverty_rate_1990   1.775647
7                                    population_1970   1.441041
22                         civilian_labor_force_2001   1.095091
6                                    population_1960   1.040445
12                           persons_in_poverty_1960   1.036477
23                         civilian_labor_force_2002   0.949674
30                         civilian_labor_force_2009   0.870865
19                         urban_influence_code_2013   0.852439
13                           persons_in_poverty_1970   0.701951
38                         civilian_labor_force_2017   0.675041
27                         civilian_labor_force_2006   0.667784
10                                   population_2000   0.667125
25                         civilian_labor_force_2004   0.665199
14                           persons_in_poverty_1980   0.605252
48                                     employed_2007   0.527751
26                         civilian_labor_force_2005   0.526194
5                                  poverty_rate_2010   0.525349
16                           persons_in_poverty_2000   0.490510
29                         civilian_labor_force_2008   0.454516
37                         civilian_labor_force_2016   0.434639
34                         civilian_labor_force_2013   0.302432
4                                  poverty_rate_2000   0.291033
28                         civilian_labor_force_2007   0.281774
39                         civilian_labor_force_2018   0.270331
9                                    population_1990   0.247925
8                                    population_1980   0.237542
21                         civilian_labor_force_2000   0.232738
42                                     employed_2001   0.231179
15                           persons_in_poverty_1990   0.228147
61                                   unemployed_2000   0.214445
46                                     employed_2005   0.207681
70                                   unemployed_2009   0.178447
59                                     employed_2018   0.177478
60                                     employed_2019   0.173603
49                                     employed_2008   0.167382
71                                   unemployed_2010   0.162577
36                         civilian_labor_force_2015   0.160169
47                                     employed_2006   0.150296
17                           persons_in_poverty_2010   0.142963
52                                     employed_2011   0.142608
63                                   unemployed_2002   0.122074
31                         civilian_labor_force_2010   0.121756
64                                   unemployed_2003   0.117289
78                                   unemployed_2017   0.112149
62                                   unemployed_2001   0.111712
40                         civilian_labor_force_2019   0.103100
72                                   unemployed_2011   0.091851
24                         civilian_labor_force_2003   0.090762
67                                   unemployed_2006   0.087287
33                         civilian_labor_force_2012   0.087218
76                                   unemployed_2015   0.078287
20                                        metro_2013   0.075213
53                                     employed_2012   0.071918
92                            unemployment_rate_2011   0.071653
93                            unemployment_rate_2012   0.057671
57                                     employed_2016   0.052072
77                                   unemployed_2016   0.051370
51                                     employed_2010   0.045957
56        

In [114]:
third_pca_df = pca_data_df[["attributes", "third_pc"]]
third_pca_df.sort_values("third_pc", ascending=False)

attributes  third_pc
1                                  poverty_rate_1970  6.948368
2                                  poverty_rate_1980  5.657152
3                                  poverty_rate_1990  5.362060
4                                  poverty_rate_2000  3.180858
8                                    population_1980  3.131666
9                                    population_1990  1.864400
13                           persons_in_poverty_1970  1.625454
5                                  poverty_rate_2010  1.594653
0                                  poverty_rate_1960  1.391390
15                           persons_in_poverty_1990  1.369300
17                           persons_in_poverty_2010  1.301316
7                                    population_1970  1.186005
6                                    population_1960  1.126271
12                           persons_in_poverty_1960  0.906968
25                         civilian_labor_force_2004  0.725800
14                           persons_in_poverty_1980  0.698369
23                         civilian_labor_force_2002  0.663215
19                         urban_influence_code_2013  0.653382
16                           persons_in_poverty_2000  0.537169
18                   rural_urban_continuum_code_2013  0.518473
21                         civilian_labor_force_2000  0.471644
27                         civilian_labor_force_2006  0.468558
10                                   population_2000  0.447446
30                         civilian_labor_force_2009  0.447221
38                         civilian_labor_force_2017  0.431190
29                         civilian_labor_force_2008  0.418610
53                                     employed_2012  0.409359
24                         civilian_labor_force_2003  0.399869
44                                     employed_2003  0.398627
43                                     employed_2002  0.387078
46                                     employed_2005  0.386910
32                         civilian_labor_force_2011  0.342780
61                                   unemployed_2000  0.337002
35                         civilian_labor_force_2014  0.321442
33                         civilian_labor_force_2012  0.304441
34                         civilian_labor_force_2013  0.295936
47                                     employed_2006  0.293327
20                                        metro_2013  0.290851
31                         civilian_labor_force_2010  0.261677
60                                     employed_2019  0.225344
26                         civilian_labor_force_2005  0.225331
59                                     employed_2018  0.215680
54                                     employed_2013  0.209844
45                                     employed_2004  0.197375
62                                   unemployed_2001  0.193251
64                                   unemployed_2003  0.180960
66                                   unemployed_2005  0.176930
58                                     employed_2017  0.169192
28                         civilian_labor_force_2007  0.160873
40                         civilian_labor_force_2019  0.155640
41                                     employed_2000  0.153267
42                                     employed_2001  0.144901
55                                     employed_2014  0.135252
49                                     employed_2008  0.134505
52                                     employed_2011  0.121212
48                                     employed_2007  0.120256
65                                   unemployed_2004  0.115810
57                                     employed_2016  0.094184
22                         civilian_labor_force_2001  0.091534
79                                   unemployed_2018  0.085308
75                                   unemployed_2014  0.083554
50                                     employed_2009  0.070688
81                            unemployment_rate_2000  0.069420
37        

In [115]:
fourth_pca_df = pca_data_df[["attributes", "fourth_pc"]]
fourth_pca_df.sort_values("fourth_pc", ascending=False)

attributes  fourth_pc
2                                  poverty_rate_1980   2.768204
4                                  poverty_rate_2000   2.007991
5                                  poverty_rate_2010   1.758616
8                                    population_1980   1.722055
3                                  poverty_rate_1990   1.606528
19                         urban_influence_code_2013   1.599778
17                           persons_in_poverty_2010   1.577903
12                           persons_in_poverty_1960   1.506804
1                                  poverty_rate_1970   1.418868
10                                   population_2000   1.368049
27                         civilian_labor_force_2006   0.933315
14                           persons_in_poverty_1980   0.909203
30                         civilian_labor_force_2009   0.907037
6                                    population_1960   0.892736
22                         civilian_labor_force_2001   0.836022
7                                    population_1970   0.598681
11                                   population_2010   0.545029
15                           persons_in_poverty_1990   0.517703
28                         civilian_labor_force_2007   0.488380
53                                     employed_2012   0.460826
55                                     employed_2014   0.415029
23                         civilian_labor_force_2002   0.411594
16                           persons_in_poverty_2000   0.375111
29                         civilian_labor_force_2008   0.370242
38                         civilian_labor_force_2017   0.368995
24                         civilian_labor_force_2003   0.359943
31                         civilian_labor_force_2010   0.309152
26                         civilian_labor_force_2005   0.308528
32                         civilian_labor_force_2011   0.299830
47                                     employed_2006   0.268733
0                                  poverty_rate_1960   0.266228
21                         civilian_labor_force_2000   0.263566
20                                        metro_2013   0.259668
58                                     employed_2017   0.258093
70                                   unemployed_2009   0.256974
50                                     employed_2009   0.248515
49                                     employed_2008   0.242882
13                           persons_in_poverty_1970   0.236531
54                                     employed_2013   0.221757
39                         civilian_labor_force_2018   0.211999
44                                     employed_2003   0.206089
33                         civilian_labor_force_2012   0.189446
60                                     employed_2019   0.177539
56                                     employed_2015   0.159143
62                                   unemployed_2001   0.154959
69                                   unemployed_2008   0.148301
41                                     employed_2000   0.146213
37                         civilian_labor_force_2016   0.138297
34                         civilian_labor_force_2013   0.136693
67                                   unemployed_2006   0.136481
65                                   unemployed_2004   0.120745
71                                   unemployed_2010   0.102635
18                   rural_urban_continuum_code_2013   0.098002
61                                   unemployed_2000   0.083318
35                         civilian_labor_force_2014   0.083013
72                                   unemployed_2011   0.078514
51                                     employed_2010   0.071955
52                                     employed_2011   0.068687
42                                     employed_2001   0.063325
73                                   unemployed_2012   0.062139
68                                   unemployed_2007   0.057158
46                                     employed_2005   0.054291
45        

In [52]:
pca.explained_variance_ratio_[:4]
#The first 8 principal components explain ~65% of the variance at the data 

array([0.39947365, 0.12950731, 0.12068972, 0.08225538])

# Running Model Using PCA

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [54]:
target = df['bellwether?']
target

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
30     0
31     0
32     0
33     0
34     0
35     0
36     0
37     0
38     0
39     0
40     0
41     0
42     0
43     0
44     0
45     0
46     0
47     0
48     0
49     0
50     0
51     0
52     0
53     0
54     0
55     0
56     0
57     0
58     0
59     0
60     0
61     0
62     0
63     1
64     0
65     0
66     0
67     0
68     0
69     0
70     0
71     0
72     0
73     0
74     0
75     0
76     0
77     0
78     0
79     0
80     0
81     0
82     0
83     0
84     0
85     0
86     0
87     0
88     0
89     0
90     0
91     0
92     0
93     0
94     0
95     0
96     0
97     0
98     0
99     0
100    0
101    0
102    0
103    0
104    0
105    0
106    0
107    0
108    0
109    0
110    0
1

In [55]:
limited_pca_4 = PCA_components.iloc[:,:4]

In [56]:
limited_pca_4

0          1          2          3
0    -2.694183  -7.661438   2.434343  -3.751954
1    -4.766801  -2.766472   6.255390  -1.775647
2    -1.391390  -6.948368   5.657152  -5.362060
3    -0.266228   1.418868   2.768204  -1.606528
4    -6.221278  -7.465476  11.200364   4.203774
5     6.531149   0.251054  -0.599539  -2.254340
6    10.075031   3.473045   2.568853  -4.019910
7     6.309566  -6.272881   3.788199  -4.072819
8    10.916391   1.352221   0.059373  -3.647745
9    -2.927851  -6.304850   1.716056  -1.878199
10   -0.075722  -1.149611  -0.961547  -2.823876
11   -1.575033  -0.056763  -0.511172   2.937421
12   -5.311812  -8.544785   0.533173  -1.579315
13   -3.225902  -4.420459  -1.573749  -2.353233
14   -2.550059  -3.769302   0.796102  -2.046164
15   -8.989388  -5.324400   0.620356  -0.331826
16   -5.667354  -7.464231   2.374368  -1.142205
17   -3.533800  -4.746964   1.624899  -1.419862
18   -8.374816  -3.441536  -3.184149  -1.671526
19    6.133051  -0.430332   0.261492  -3.183571
20   -3.263504  -3.553092  -1.910580  -2.393960
21   -1.197866  -2.214633  -2.123762  -2.257438
22   -0.177873  -4.141192  -0.693782  -2.511512
23   -7.288345  -5.063365  -0.434861  -1.665256
24   -8.853015  -9.459260   3.859569   0.701991
25   -7.281674  -7.792788  -0.067960  -1.872561
26   -3.853301  -4.468735  -1.955875  -1.762827
27   -5.467974  -2.205613  -3.078439   1.986033
28   -9.501549  -6.196728   0.241320  -0.272468
29   -7.842437  -3.762043  -2.618294  -0.690174
30    7.842445   0.530867  -1.498814  -4.808888
31   -0.147383  -0.740665  -3.739738  -9.724727
32   11.690662   4.828371   0.345686   3.732694
33  -18.162063  -0.762601   8.122872  11.937060
34   10.188383  -6.126869   8.526644   2.184539
35   -1.387090  -0.811423   1.431883  -3.625798
36   14.623016  11.124162   1.404724  -1.820236
37   -1.274079   0.320287   9.412293  12.103919
38   -6.064179  -3.525229   9.894604   9.066640
39   11.655730   4.994334   6.582494   8.572725
40    9.337529   4.363416   8.135596  10.135071
41   16.733603   5.031435   5.950678   6.766167
42   -1.142959  -2.051402   7.518643   2.694439
43   17.986767   5.050540   5.978516   3.418455
44   10.770232   0.204565   1.794177  -6.453611
45   18.075533   3.555002   6.489688   4.752744
46   -5.763020  -3.394475   4.064816  -2.571189
47   18.075357  -6.509947   9.844713   1.117118
48   16.533009   3.601403  -1.405111  -4.842848
49   18.492037   9.197887  -2.084266  -7.070773
50   -7.097654   2.354177  -3.113604  -0.359483
51   -9.121010  -1.945205  -0.068869   0.772630
52   12.879944  10.180067  -4.788417  -2.955625
53   -1.880472   6.104811  -4.984401  -0.739673
54    2.048625   0.513336   4.737538   5.098641
55   -4.588786   7.651568  -3.537539   1.734776
56  -21.614359   6.679033  -2.513595  10.614354
57   -9.878200  -3.145347   1.864592  -0.740702
58   -3.918090   6.296956  -6.399602  -3.870128
59    1.677478   1.584275   1.732035   3.843858
60  -12.272568   6.677845  -3.723113   1.525763
61  -14.180260   7.180859  -9.251179  -3.735016
62    0.482580  10.588039  -1.273517   5.940399
63   22.390203   2.714914   4.244391  -1.508376
64    9.423688   0.830654   0.823651  -2.084559
65    5.198414  -0.272400   0.559160   1.387249
66    3.162991  -3.260553   5.843241   7.963023
67   -8.162627  -6.180164  -1.170967  -0.860758
68   18.367109   2.733580   1.304850  -0.890415
69   14.905696   2.707275   2.456894   1.542846
70   10.218300   2.734806   3.289409   2.837662
71   28.570601   2.688466   6.013810   2.034947
72   24.327364   2.951757   4.307466  -0.252771
73   11.940993   4.760419   1.427838   4.717807
74   -8.316860  -2.047338  -3.886542  -1.515626
75   17.347180   0.671063   1.833574  -3.106340
76    6.461688  -5.235191  -0.466953  -1.971372
77    4.909526  -1.931734   3.476379   4.476482
78   -2.631830  -7.062955   0.935551  -1.708303
79   -6.936303  -4.903743   3.167204   9.145910
80   -5.470185  -6.849356  -0.867077  -0.900838
81   -6.064405  -5.715009  -0.118126   4.445918
82   -3.251243  -7.007279   0

In [57]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(limited_pca_4, target, test_size=0.3, random_state=42)

# Oversample

In [58]:
oversample = RandomOverSampler(sampling_strategy=0.5)
X_over_train, y_over_train = oversample.fit_resample(X_train, y_train)

In [59]:
model = RandomForestClassifier(max_depth=2, random_state=0)

In [60]:
model.fit(X_over_train, y_over_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [61]:
x = pd.DataFrame(y_over_train)
x.groupby('bellwether?').size()

bellwether?
0    398
1    199
dtype: int64

In [62]:
199 / (199 + 398)

0.3333333333333333

In [63]:
y_pred_oversample = model.predict(X_test)

In [64]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

#print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred_oversample))
print(precision_score(y_test,y_pred_oversample))
print(recall_score(y_test,y_pred_oversample))
print(confusion_matrix(y_test,y_pred_oversample))

0.8936170212765957
0.0
0.0
[[168   4]
 [ 16   0]]


# Without Oversample

In [65]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [66]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [67]:
y_pred = clf.predict(X_test)

In [68]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))

[[172   0]
 [ 16   0]]
0.9148936170212766
0.0
0.0


C:\Users\AnshK\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
#baseline accuracy
570 / (56 + 570)

0.9105431309904153